In [1]:
import sys
import os

sys.path.append(os.path.abspath("../"))

In [25]:
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingOneHot
from decomposition.qhyper.algorithm import WorkflowDecompositionQHyperAdapter
from decomposition.qhyper.solver import DecomposedWorkflowSchedulingSolver, WorkflowSchedulingSolverDecorator
from QHyper.solvers import Gurobi

In [11]:
def deadline_as_cpv(tasks_file, machines_file):
    workflow = Workflow(tasks_file, machines_file, 100000)
    mean_times = workflow.time_matrix.mean(axis=1).to_dict()

    def path_load(p):
        return sum([mean_times[t] for t in p])

    return int(max(path_load(p) for p in workflow.paths))

In [12]:
tasks_file = "resources/workflows/1000genome-chameleon-6ch-250k-001.json"
machines_file = "resources/machines/linear_smaller_diff.json"
deadline = deadline_as_cpv(tasks_file, machines_file)

workflow = Workflow(tasks_file, machines_file, deadline)

In [13]:
print(len(workflow.wf_instance.workflow.tasks))

246


In [26]:
division = WorkflowDecompositionQHyperAdapter(workflow).decompose(max_subgraph_size=4)
problems = map(lambda w: WorkflowSchedulingOneHot(w), division.workflows)
solvers = map(lambda p: WorkflowSchedulingSolverDecorator(Gurobi(p)), problems)
solver = DecomposedWorkflowSchedulingSolver(list(solvers), division)

In [27]:
solution = solver.solve()

Set parameter Username


GurobiError: License expired 2024-11-12